In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("housing.csv")

In [3]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [9]:
import numpy as np

df["income_cat"] = pd.cut(
    df["median_income"],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)

In [11]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df["income_cat"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

In [13]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [15]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [17]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

housing_num = housing.drop("ocean_proximity", axis=1)

imputer.fit(housing_num)
housing_num = imputer.transform(housing_num)

In [19]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[["ocean_proximity"]]

encoder = OneHotEncoder()
housing_cat_encoded = encoder.fit_transform(housing_cat)

In [21]:
from scipy.sparse import hstack

housing_prepared = hstack([housing_num, housing_cat_encoded])

In [25]:
housing_num, housing_cat_encoded

(array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
          7.0600e+02,  2.1736e+00],
        [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
          7.6800e+02,  6.3373e+00],
        [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
          3.0000e+02,  2.8750e+00],
        ...,
        [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
          1.7200e+02,  3.1797e+00],
        [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
          5.0100e+02,  4.1964e+00],
        [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
          1.9700e+02,  3.1319e+00]]),
 <16512x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 16512 stored elements in Compressed Sparse Row format>)